# FIT5196-S1-2019 Assessment 1 TASK 2
### Student Name: Rohan Singh
### Student I.D: 30042496
#### Date:12th April,2019
#### Environment: Python 3.6 and Jupyter notebook
#### Libraries used: 

* nltk 3.2.2 (Natural Language Toolkit, included in Anaconda Python 3.6)
* nltk.collocations (for finding bigrams, included in Anaconda Python 3.6)
* nltk.tokenize (for tokenization, included in Anaconda Python 3.6)
* re  (for regular expression, included in Anaconda Python 3.6)
* Itertools (to chain all tokens in a list )

### 1. Introduction

This assessment touches on the next step of analyzing textual data, i.e., converting the extracted
data into a proper format. In this assessment, we write Python code to
preprocess a set of unit information and convert them into numerical representations

### 2. Sequence of tasks performed on the given text:

Here is sequence of tasks performed on the given list of 200 units:
<n><b>1.</b> Tokenization
<n><b>2.</b> Case normalisation
<n><b>3.</b> Finding Bigrams
<n><b>4.</b> Retokenize Bigrams
<n><b>5.</b> Removal of stopwords and other words (length<3,<5% & >95% occurence)
<n><b>6.</b> Porter Stemming
<n><b>7.</b> Getting text output

### 3. Code Methodology

##### <u>Importing libraries

In [1]:
#Importing Libraries 

import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.util import ngrams
from nltk.collocations import *
from nltk.probability import *
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from itertools import chain
import itertools

##### <u>Extracting Data from text file

Here we extract the data using split method. It allows us to consider each unit as a single separate document.
We append Keys list with the name of each unit and the values list as their corresponding content. Each of the pair of unit and its description is zipped together as a tuple and a list of tuples is prepared (patents_raw).

*Note: <u>We didn't use dictionary since there was duplication of units in the keys list. Hence we opted for list of tuples to retain each element.

In [2]:
filename = '30042496_task2.txt'
splitter=']'
keys_list=[]
values_list=[]

with open(filename) as f:
    
    for chunk in f.read().split(splitter):
        unit = re.findall("[A-Z]{3,4}\d{4}",chunk)
        if len(unit)>=1:
            keys_list.append(unit[0])
            values_list.append(chunk)

values_list = [s.replace('Title','') for s in values_list]
values_list = [s.replace('Synopsis','') for s in values_list]
values_list = [s.replace('Outcomes','') for s in values_list]

patents_raw=list(zip(keys_list,values_list))


##### <u>Tokenization & Case Normalization

We define a function named <b>tokenizePatent(unit)</b>, which takes the input as unit code and tokenizes its corresponding content in the tuple list using the given regular expression. The function also performs case normalisation as shown in the code comment below, where it keeps all the capital tokens unchanged and converts the rest to lowercase.

We finally have a list of tuples with each tuple in the list as (unit,[tokens]). This list is named as patents_tokenized.


In [3]:
tokenizer=RegexpTokenizer(r"\w+(?:[-']\w+)?")

def tokenizePatent(unit): 
        raw_patent_list=[element[1] for element in patents_raw if unit==element[0]]
        for raw_patent in raw_patent_list:
            tokenized_patents = tokenizer.tokenize(raw_patent)
            tokenized_patents= [token  if token.isupper() else token.lower() for token in tokenized_patents] #Case Normalisation
            return (unit, tokenized_patents) # returns a tuple of unit code and a list of tokens
    
patents_tokenized = list(tokenizePatent(unit) for unit in keys_list)

# print(patents_tokenized)

#### <u>Finding Bigrams

We make use of nltk.collocations library to gather all the meaningful bigrams with respect to PMI index. Also we apply a filter that excludes any bigram less than 3 words.

In [4]:
all_words = list(chain.from_iterable(element[1] for element in patents_tokenized)) #collects all unit details in one list

bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(all_words)
bigram_finder.apply_word_filter(lambda w: len(w) < 3)  #To exclude any bigram less than 3 words
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) 
# top_200_bigrams

#### <u> Retokenizing the bigrams
    
 The bigrams are retokenized and stored in a list of tuples named colloc_patents. The list colloc_patents have tuples having data in the form of (unit,[Multiword tokens]).

In [5]:
mwetokenizer = MWETokenizer(top_200_bigrams)
colloc_tokens=[mwetokenizer.tokenize(element[1]) for element in patents_tokenized]
colloc_patents=list(zip(keys_list,colloc_tokens))    


#### <u>Removing stopwords and words with length less than 3

We will now remove the stopwords from our tokens. These are the words which increase the size of vocabulary but add very little or no meaning to it.For example: a, a's, am, aren't, do, is,...etc.

In [6]:
stopwords=open('stopwords_en.txt',"r",encoding="utf8")  
stopwords_list=stopwords.read().split('\n')
for i in range(len(colloc_patents)): 
    colloc_patents[i]=list(colloc_patents[i]) #convert to list since tuples are immutable
    colloc_patents[i][1]= [word for word in colloc_patents[i][1] if word not in stopwords_list] #Removing stopwords that are in stopwords_en.txt
    colloc_patents[i][1]= [word for word in colloc_patents[i][1] if len(word)>=3] #Removing words ith length less than 3
    colloc_patents[i][1]=list(set(colloc_patents[i][1]))  #To keep unique tokens for every unit
    colloc_patents[i]=tuple(colloc_patents[i])  #convert back to tuple 
    

#### <u> Removing words that occur in <5% and >95%
 
Here we collect all the unique tokens from each unit into a list all_words_colloc and pass this through a function countX, which returns a tuple with the word and its count. We store each tuple of (word, frequency) in a dictionary named w_freq, which has only unique tuples.So w_freq has items in the form {word1:frequency, word2:frequency,.....}.

Finally we iterate over this dictionary to remove the words that occur in less than 5% and more than 95% documents.
<n> <b>% of occurence=100*(word frequency)/(Total No. of Documents)

In [7]:
def countX(lst, x):                    #returns a tuple with the word and its count
    return (x,lst.count(x)) 

all_words_colloc = list(chain.from_iterable(element[1] for element in colloc_patents))

w_freq = dict(set([countX(all_words_colloc, x) for x in all_words_colloc]))  #to get unique tuples of (word,frquency)


remove5_percent=[]                                      # Removal list for less than 5%
for k,v in w_freq.items():
    if v/len(colloc_patents)<0.05:
        remove5_percent.append(k)

remove95_percent=[]                                        # Removal list for more than 95%
for k,v in w_freq.items():
    if v/len(colloc_patents)>0.95:
        remove95_percent.append(k)

final_removal_list = remove5_percent + remove95_percent      #combining elements of both the lists into one final list

for i in range(len(colloc_patents)): 
    colloc_patents[i]=list(colloc_patents[i])    #convert to list since tuples are immutable
    colloc_patents[i][1]= [word for word in colloc_patents[i][1] if word not in final_removal_list]
    colloc_patents[i]=tuple(colloc_patents[i])  #convert back to tuple

print(len(remove5_percent))     #length of words with less than 5% occurence
print(len(remove95_percent))    #length of words with more than 95% occurence




3418
0


#### <u> Porter Stemming

Stemming is the process of reducing inflection in words to their root forms such as mapping a group of words to the same stem even if the stem itself is not a valid word in the Language.Finally we perform Porter stemming on the tokens extracted.

In [8]:
ps = PorterStemmer()               #Stemming using Porter Stemmer
for i in range(len(colloc_patents)): 
    colloc_patents[i]=list(colloc_patents[i])  #convert to list since tuples are immutable
    colloc_patents[i][1]= [ps.stem(word) for word in colloc_patents[i][1]]
    colloc_patents[i]=tuple(colloc_patents[i])  #convert back to tuple

# print(colloc_patents)

#### Generating Outputs

This is the last step, where we generate sparse representations for the unit information and get the vocab list with token index. The list of vocabulary is stored in <b>30042496_vocab.txt</b> while the sparse representation is stored in <b> 30042496_countVec.txt

In [9]:
colloc_voc = list(set(chain.from_iterable(element[1] for element in colloc_patents)))       
colloc_voc.sort()   #Sorts the Vocab alphabetically
out_file1 = open("30042496_vocab.txt", 'w')       # creating two output files
out_file2=open('30042496_countVec.txt','w')

colloc_dict = {}

i = 0                              #counter for indexing tokens
for w in colloc_voc:
    colloc_dict[w] = i      
    i = i + 1                                      
for k,v in colloc_dict.items():
    out_file1.write("{}:{}\n ".format(k,v))  #to print the vocab in desired format
    
for element in colloc_patents:
    out_file2.write(element[0]+' ,')
    dict_index = [colloc_dict[w] for w in element[1]]
    size=len(set(dict_index))
    for k, v in FreqDist(dict_index).items():
        size-=1
        if size==0:
            out_file2.write("{}:{} ".format(k,v))  #to print the sparse representation in desired format (removes',' from end)
        else:
            out_file2.write("{}:{}, ".format(k,v))
        
    out_file2.write('\n')
out_file1.close()   #closing both files
out_file2.close()
print(colloc_voc)

['abil', 'academ', 'achiev', 'acquir', 'activ', 'address', 'advanc', 'aim', 'analys', 'analysi', 'analyt', 'appli', 'applic', 'apprais', 'approach', 'area', 'argument', 'articul', 'aspect', 'assess', 'australia', 'australian', 'awar', 'base', 'basi', 'basic', 'behaviour', 'bodi', 'build', 'busi', 'capac', 'care', 'case', 'challeng', 'chang', 'clinic', 'collect', 'common', 'commun', 'compet', 'complex', 'concept', 'conceptu', 'conduct', 'consid', 'construct', 'contemporari', 'content', 'context', 'control', 'core', 'cover', 'creation', 'creativ', 'critic', 'critiqu', 'cultur', 'current', 'data', 'deal', 'debat', 'deliveri', 'demonstr', 'describ', 'design', 'develop', 'disciplin', 'discuss', 'divers', 'econom', 'effect', 'emphasi', 'engag', 'engin', 'enhanc', 'environ', 'environment', 'ethic', 'evalu', 'evid', 'examin', 'expect', 'experi', 'experiment', 'explain', 'explor', 'factor', 'featur', 'field', 'find', 'focu', 'form', 'formul', 'framework', 'fundament', 'futur', 'gain', 'global',